# Mini NeRF

## Introduction

Novel view synthesis an important problem in computer vision. It has a lot of applications such as video enhancement and virtual reality. *Neural Radiance Fields* (NeRFs) is a simple and powerful model for such kind of problem. It use the idea of **volume rendering** to train a model that implicitly contains the 3D information and samples from it to reconstruct the image. By encoding both position and viewing direction into the input, the NeRF model is able to "understand" the radiance distribution in the scene, thus reconstructing the image with better illumination details.

In this project, we reimplemented the NeRF method using pytorch and train it with smaller MLP and lower resolution images using only the "coarse" model so that it can quickly converge on smaller GPU.

## Setting up environments

In [ ]:
# imports
import torch
import torch.nn.functional as F
import numpy as np
import sys
import os
from tqdm import tqdm
from matplotlib import pyplot as plt

# select device
DEVICE = None
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Bad to go!")
    DEVICE = torch.device("cpu")
# load config files
sys.path.append(os.getcwd())
import configs.hotdog, configs.lego
dataset_configs:list = [configs.hotdog, configs.lego]

In [ ]:
# hyper parameters for model
sample_t: tuple = (2,6) # t_n, t_f

# 100x100: scale_factor=3
# 200x200:scale_factor=2
scale_factor = 3

# dataset to use
config = configs.lego

## Dataset

We are using two scenes from the datasets of the NeRF paper: the lego model and the hotdog. These datasets are generated from Blender so that the ground truth intrinsic and extrinsic parameters are known. The images are resized to 100x100 for smaller GPU. 

Below are examples extracted from the dataset. We can see that the lego model contains many details while the hotdog model is simpler but contains rich illumination features.

In [ ]:
from nerf.data import load_blender
imgs_dict:dict = {}
poses_dict:dict = {}
int_mat_dict:dict = {}
demo_images: list = []
names: list = []
for data_config in dataset_configs:
    key:str = data_config.expname
    imgs_dict[key], poses_dict[key], int_mat_dict[key] = load_blender(data_config.datadir, device="cpu", scale_factor=scale_factor)
    demo_images.append(imgs_dict[key][0])
    names.append(data_config.expname)

# plot images
fig = plt.figure(figsize=(12,10))
columns = 2
for i, image in enumerate(demo_images):
    ax = plt.subplot(int(len(demo_images) / columns + 1), int(columns), i + 1)
    ax.title.set_text(names[i])
    plt.imshow(image)
plt.show()

# rename data for further usage
imgs, poses, int_mat = imgs_dict[config.expname], poses_dict[config.expname], int_mat_dict[config.expname]
img_h, img_w = imgs.shape[1:3]

## Method
In this section, we showed several functions used during training and inference, and tested them with fake data input.

### Ray Generation
This function generate a vector indicating a ray from each of the pixels of an image and transform them into the world coordinate using camera extrinsics.

In [ ]:
# compute rays
from nerf.graphics import compute_rays

rays_o, rays_d = compute_rays((img_h, img_w), int_mat, poses[0])
print("origin: ", rays_o[0,0])
print("center of ray: ", rays_d[img_h//2,img_w//2])

### Sampling from ray

This function generate samples from rays using the coarse sampling method described from the paper: divide the ray in a range to equally size bins, sample uniformly from each bin to get a set of sample points. For training time consideration, we did not implemented the fine model.

The test block extracted samples from one ray, we can see that the samples are distributed on a line.

In [ ]:
from nerf.graphics import queries_from_rays
samples, depths = queries_from_rays(rays_o, rays_d, sample_t, 8)
print("samples[0, 0]: ", samples[0,0])
print("depths[0, 0]: ", depths[0,0])

### Positional Encoding

The author of the paper found that the model will be very hard to converge if the positions and directions are passed directly into the model. Therefore, a encoding function is needed to map the input to higher dimension space.

In [ ]:
from nerf.nerf_helper import PosEncode

L = 6
x = torch.tensor([[ 1.8013, -0.6242,  0.7009]])
enc_x = PosEncode(x, L, True)
print(enc_x)

### Rendering
The NeRF model will output the rgb values and the volume densities at the sampled points. A rendering function is needed to combine these results together and calculate the final rgbd value displaying on the pixel.

The testing block use the original training image as an fake output of the MLP and the render function successfully recovered the image.

In [ ]:
from nerf.graphics import render_from_nerf
fake_depth = torch.Tensor([1])
fake_nerf_output = imgs[0].cpu().reshape(img_h, img_w, 1, 4)
rgb, depth = render_from_nerf(fake_nerf_output, fake_depth)
plt.imshow(rgb)
plt.title("Recovered image")
plt.show()

### Generating poses
This function is used when we want to view the model from a new perspective (pose).

In [ ]:
from nerf.graphics import generate_demo_poses
demo_poses = generate_demo_poses()
print(demo_poses.shape)
print(demo_poses[0])

## Experiment
In this section we will show a demo process of training and compare the pretrained models.

### Training

In [ ]:
from nerf.model import NeRF 
import os.path

seed = 9458
torch.manual_seed(seed)
np.random.seed(seed)
###### hyper-parameters
L_pos = 10
L_dir = 4
num_samples = 32
batch_size = 8192 # increase batchsize if u have large GPU MEM
fc_width = 128
fc_depth = 4
skips = [2]
lr = 5e-4
num_it = 10001
display_every = 200

###### models
model = NeRF(ch_in_pos=6*L_pos+3, ch_in_dir=6*L_dir+3, fc_width=fc_width, fc_depth=fc_depth, skips=skips)
model.to(DEVICE)

###### load validation data
imgs_val, poses_val, _ = load_blender(config.datadir, data_type="val",scale_factor=scale_factor, device="cpu")
num_val = imgs_val.shape[0]

###### train
psnrs = []
val_its = []
i = 0

###### optimizer, checkpoint
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=lr)
ckpt_path = 'nerf.pt'

###### check saved checkpoints
if os.path.exists(ckpt_path):
    print("checkpoint found! Loading...")
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    i = checkpoint['epoch']
    loss = checkpoint['loss']
    psnrs = checkpoint['psnrs']
    val_its = checkpoint['its']
    print("checkpoint loaded, i =",i)
else:
    print("No checkpoint found")

In [ ]:
# start training
from nerf.training_logic import train_NeRF
train_NeRF(model = model, optimizer=optimizer,imgs_train=imgs, imgs_val=imgs_val, poses_train=poses, poses_val=poses_val,int_mat=int_mat, sample_t=sample_t,
            L_pos=L_pos, L_dir=L_dir, num_samples=num_samples, ckpt_path=ckpt_path, batch_size=batch_size,
            psnrs=psnrs, val_its=val_its, start_iter_num=i, end_iter_num=num_it)

### Inference and generate video
After training we can generate a demo video using generated pose.

In [ ]:
from torch import Tensor
from nerf.nerf_helper import nerf_iter_once
import cv2
import imageio
# generate pose
model.eval()
gen_num: int = 120
repeat: int = 2
gen_poses:Tensor = generate_demo_poses(height=4, num_poses=gen_num).to(poses).to(DEVICE)
gen_imgs:list = []
for i in range(gen_num):
    with torch.no_grad():
        pred_rgb, pred_depth = nerf_iter_once(
                model,
                (img_h, img_w),
                int_mat.to(DEVICE),
                gen_poses[i],
                sample_t,
                L_pos,
                L_dir,
                num_samples=num_samples,
                batch_size=batch_size
                )
    # concat channels
    pred_rgbd: Tensor = torch.cat([pred_rgb, pred_depth[...,None]], dim=-1)
    # translate to [0,255]
    img_np = np.array(pred_rgbd.detach().cpu()*255).astype(np.uint8)
    img_np = cv2.resize(img_np, (112,112), interpolation=cv2.INTER_AREA)
    gen_imgs.append(img_np)

gen_imgs = gen_imgs * repeat

demo_video_path: str = "media/{}.mp4".format(config.expname)
imageio.mimwrite(demo_video_path, gen_imgs, fps=30, quality=8)

In [ ]:
from ipywidgets import Video
video = Video.from_file(demo_video_path)
video

### Test and Comparison
We perform inference based on the poses of the test images and compare the results with the ground truth. The results aer shown below.

lego_100x100 on testdata:
PSNR:  25.975324880729126
SSIM:  0.8976107119105973

Hotdog on test dataset:
PSNR:  29.81981982968555
SSIM:  0.9356882317837466

In [ ]:
import pytorch_msssim
def test():
    psnr_ = 0
    ssim_ = 0
    for i in range(num_test):
        test_idx = i
        test_img = imgs_test[test_idx].clone().to(DEVICE)
        test_c2w = poses_test[test_idx].clone().to(DEVICE)

        pred_rgb, _ = nerf_iter_once(
                    model,
                    (img_h, img_w),
                    int_mat_test.to(DEVICE),
                    test_c2w,
                    sample_t,
                    L_pos,
                    L_dir,
                    num_samples=num_samples,
                    batch_size=batch_size
                )
        loss = torch.nn.functional.mse_loss(pred_rgb, test_img[..., :3])

        psnr_ += float(-10. * torch.log10(loss))
        
        ssim_val = pytorch_msssim.ssim(pred_rgb.permute(2,0,1).reshape(1,3,200,200), test_img[..., :3].permute(2,0,1).reshape(1,3,200,200), data_range=1)
        ssim_ += float(ssim_val)

    psnr_ /= num_test
    ssim_ /= num_test

    print("PSNR: ", psnr_)
    print("SSIM: ", ssim_)

    
imgs_test, poses_test, int_mat_test = load_blender(config.datadir, data_type="test",scale_factor=scale_factor, device="cpu")
num_test = imgs_val.shape[0]

with torch.no_grad():
    test()

Compare results with ground truth image

In [ ]:
from nerf.nerf_helper import compare
data_pac:dict = {
        "imgs": imgs,
        "poses": poses,
        "DEVICE": DEVICE,
        "config": config,
        "scale_factor": scale_factor,
        "sample_t": sample_t,
        "num_samples": num_samples,
        "batch_size": batch_size,
        "model": model,
        "L_pos": L_pos,
        "L_dir": L_dir
}
ckpt_path = './trained_models/basic_nerf_hotdog_9800.pt'
if os.path.exists(ckpt_path):
    print(ckpt_path, "found! Loading...")
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    i = checkpoint['epoch']
    loss = checkpoint['loss']
    psnrs = checkpoint['psnrs']
    its = checkpoint['its']
    print("checkpoint loaded, i =",i)
else:
    print("No checkpoint found")
    
with torch.no_grad():
    compare(data_pac)

![](./media/lego_cmp.png)

### Conclusion

In this project, we've reimplemented the basic functionality of NeRF in pyTorch, applied it to two scenarios and done both quantitively and qualitative analysis on the result.



The major shortcoming of this project is the lack of hierarchical sampling and the much lower resolution we select.  If given more time and computation resources, we would consider to fully reimplement NeRF, train it with standard resolution, and even try something more interesting, like [NeRF without Neural Networks](https://alexyu.net/plenoxels/).